In [369]:
import pandas as pd
import gspread
import requests

token = 'AF5y90PW7vmJALond4SEGVBFwmt3Jhj7Ba7oclAeXL4'

In [370]:
sa = gspread.service_account(filename='geprek-wani-abcd00886732.json')

In [371]:
sheet = sa.open('Wani Spreadsheet')

In [372]:
database = sheet.worksheet('Database Harga')

In [373]:
df_harga = pd.DataFrame(database.get_all_records())

In [374]:
worksheet = sheet.worksheet('Pesanan')

In [375]:
df = pd.DataFrame(worksheet.get_all_records())

In [376]:
df.rename(columns={'Request cabe GEPREK atau cabe TERIYAKI': 'Cabe', 
                   'Sambal Tambahan (Rp 2.000,-)': 'Sambal',
                   'Camilan (Rp 10.000,-)': 'Camilan',
                   }, inplace=True)

In [377]:
df.head()

,Timestamp,Nama,Makanan Utama,Cabe,Camilan,Minuman,Sambal,"Bukti Pembayaran, transfer ke BCA: 1500952195 a/n BILLY JASON MARGIANTO","Tambah Nasi Putih (Rp 5.000,-)"
0,26/05/2023 08:55:04,Aiffah,"Nasi Ayam Crisbar ~ Rp 15.000,-",,,,,https://drive.google.com/open?id=1yxKPDDxjp8um...,
1,26/05/2023 08:46:50,Timmy,"Nasi Goreng Kulit ~ Rp 15.000,-",,,,,https://drive.google.com/open?id=1E44INDD2r1lC...,
2,26/05/2023 08:49:43,Max,"Indomie Ayam Geprek ~ Rp 15.000,-",,,,,https://drive.google.com/open?id=13gHMbv9x4R9j...,
3,26/05/2023 08:50:00,faza,"Nasi Ayam Crisbar ~ Rp 15.000,-",,,,,https://drive.google.com/open?id=1kc4lsFiRn_Is...,
4,26/05/2023 08:51:04,Irfan dary s,"Nasi Goreng Kulit ~ Rp 15.000,-",,,,,https://drive.google.com/open?id=17RcLdrmW-HPs...,


In [378]:
# create a list containing 'Makanan Utama' and 'Nama'
list = ['Makanan Utama', 'Cabe', 'Nama']

# replace the empty string with 1 if Makanan Utama is not empty
df['Cabe'] = df['Cabe'].apply(lambda x: 1 if x == '' else x)

# remove everything after ' ~' in Makanan Utama
df['Makanan Utama'] = df['Makanan Utama'].apply(lambda x: x.split(' ~')[0])
df['Minuman'] = df['Minuman'].apply(lambda x: x.split(' ~')[0])

df[list]

,Makanan Utama,Cabe,Nama
0,Nasi Ayam Crisbar,1,Aiffah
1,Nasi Goreng Kulit,1,Timmy
2,Indomie Ayam Geprek,1,Max
3,Nasi Ayam Crisbar,1,faza
4,Nasi Goreng Kulit,1,Irfan dary s
5,,1,Verill
6,Nasi Ayam Crisbar,1,Vania
7,Nasi Ayam Teriyaki,3,Kevin Sander Utomo
8,Nasi Goreng Telor,1,BilGus
9,Nasi Ayam Crisbar,1,Novia


In [379]:
# Custom geprek
df_geprek = df[df['Makanan Utama'].str.contains('Geprek', case=False) & ~df['Makanan Utama'].str.contains('Indomie') & ~df['Makanan Utama'].str.contains('Udang')]
df_geprek

,Timestamp,Nama,Makanan Utama,Cabe,Camilan,Minuman,Sambal,"Bukti Pembayaran, transfer ke BCA: 1500952195 a/n BILLY JASON MARGIANTO","Tambah Nasi Putih (Rp 5.000,-)"
16,26/05/2023 09:05:46,Rey,Ayam Geprek tanpa Nasi,0,,,,https://drive.google.com/open?id=1qnax-YPwZnft...,


In [392]:
# Dataframe not-geprek but include 'Indomie' and 'Udang' and doen't include 'Teriyaki'
df_not_geprek = df[~df['Makanan Utama'].str.contains('geprek', case=False)]
# drop rows that contains 'Teriyaki'
df_not_geprek = df_not_geprek[~df_not_geprek['Makanan Utama'].str.contains('Teriyaki')]

# drop rows with empty Makanan Utama
df_not_geprek = df_not_geprek[df_not_geprek['Makanan Utama'] != '']

# Dataframe geprek tanpa nasi
df_geprek_tanpa_nasi = df_geprek[df_geprek['Makanan Utama'].str.contains('tanpa Nasi')]

# Dataframe geprek dengan nasi
df_geprek_dengan_nasi = df_geprek[~df_geprek['Makanan Utama'].str.contains('tanpa Nasi')]

# Dataframe indomie geprek
df_indomie_geprek = df[df['Makanan Utama'].str.contains('Indomie') & df['Makanan Utama'].str.contains('Geprek')]

# Dataframe udang geprek
df_udang_geprek = df[df['Makanan Utama'].str.contains('Udang') & df['Makanan Utama'].str.contains('Geprek')]

# Dataframe camilan
df_camilan = df[['Camilan', 'Nama']]
df_camilan = df_camilan[df_camilan['Camilan'] != '']

# Dataframe minuman
df_minuman = df[['Minuman', 'Nama']]
df_minuman = df_minuman[df_minuman['Minuman'] != '']

# Dataframe sambal
df_sambal = df[['Sambal', 'Nama']]
df_sambal = df_sambal[df_sambal['Sambal'] != '']

# Dataframe teriyaki
df_teriyaki = df[df['Makanan Utama'].str.contains('Teriyaki')]

In [393]:
combined_prints = []
combined_prints.append(f"Pesanan Depot Wani: {pd.Timestamp.now()}")
combined_prints.append("=====================================")
combined_prints.extend([f"{sum(df_geprek_dengan_nasi['Cabe'] == i)} Nasi Ayam Geprek Cabe {i} ({', '.join(df_geprek_dengan_nasi[df_geprek_dengan_nasi['Cabe'] == i]['Nama'])})" for i in df_geprek_dengan_nasi['Cabe'].unique()])
combined_prints.extend([f"{sum(df_teriyaki['Cabe'] == i)} Nasi Ayam Teriyaki Cabe {i} ({', '.join(df_teriyaki[df_teriyaki['Cabe'] == i]['Nama'])})" for i in df_teriyaki['Cabe'].unique()])
combined_prints.extend([f"{sum(df_geprek_tanpa_nasi['Cabe'] == i)} Ayam Geprek tanpa nasi Cabe {i} ({', '.join(df_geprek_tanpa_nasi[df_geprek_tanpa_nasi['Cabe'] == i]['Nama'])})" for i in df_geprek_tanpa_nasi['Cabe'].unique()])
combined_prints.extend([f"{sum(df_not_geprek['Makanan Utama'] == i)} {i} ({', '.join(df_not_geprek[df_not_geprek['Makanan Utama'] == i]['Nama'])})" for i in df_not_geprek['Makanan Utama'].unique()])
combined_prints.extend([f"{sum(df_indomie_geprek['Cabe'] == i)} {df_indomie_geprek[df_indomie_geprek['Cabe'] == i]['Makanan Utama'].iloc[0]} Cabe {i} ({', '.join(df_indomie_geprek[df_indomie_geprek['Cabe'] == i]['Nama'])})" for i in df_indomie_geprek['Cabe'].unique()])
combined_prints.extend([f"{sum(df_udang_geprek['Cabe'] == i)} {df_udang_geprek[df_udang_geprek['Cabe'] == i]['Makanan Utama'].iloc[0]} Cabe {i} ({', '.join(df_udang_geprek[df_udang_geprek['Cabe'] == i]['Nama'])})" for i in df_udang_geprek['Cabe'].unique()])
combined_prints.extend([f"{sum(df_camilan['Camilan'] == i)} {i} ({', '.join(df_camilan[df_camilan['Camilan'] == i]['Nama'])})" for i in df_camilan['Camilan'].unique()])
combined_prints.extend([f"{sum(df_minuman['Minuman'] == i)} {i} ({', '.join(df_minuman[df_minuman['Minuman'] == i]['Nama'])})" for i in df_minuman['Minuman'].unique()])
combined_prints.extend([f"{sum(df_sambal['Sambal'] == i)} {i} ({', '.join(df_sambal[df_sambal['Sambal'] == i]['Nama'])})" for i in df_sambal['Sambal'].unique()])

print('\n'.join(combined_prints))

Pesanan Depot Wani: 2023-05-26 10:00:13.986438
1 Nasi Ayam Teriyaki Cabe 3 (Kevin Sander Utomo)
1 Nasi Ayam Teriyaki Cabe 5 (Awang)
1 Nasi Ayam Teriyaki Cabe 2 (Uddin)
3 Nasi Ayam Teriyaki Cabe 1 (MW, Probo, Tot)
1 Nasi Ayam Teriyaki Cabe 0 (Jeremy)
1 Ayam Geprek tanpa nasi Cabe 0 (Rey)
6 Nasi Ayam Crisbar (Aiffah, faza, Vania, Novia, Reyner Fernaldi, Galih)
2 Nasi Goreng Kulit (Timmy, Irfan dary s)
1 Nasi Goreng Telor (BilGus)
2 Nasi Ayam Richeese (Reyner Fernaldi, MC)
1 Nasi Goreng Spesial (Alex)
1 Indomie Ayam Geprek Cabe 1 (Max)
1 Indomie Telur Geprek Cabe 0 (Rey)
1 Kulit Ayam Crispy tanpa bumbu (Verill)
1 Kulit Ayam Crispy (Uddin)
1 Tahu Crispy (Jeremy)
2 Es Teh Jumbo (Rey, Tot)
2 Es Teh (Galih, Probo)
2 Sambal Bawang (Verill, Galih)


In [382]:
# remove all text after '(' in combined_prints
combined_prints = [i.split('(')[0] for i in combined_prints]

print('\n'.join(combined_prints))

Pesanan Depot Wani: 2023-05-26 09:56:02.222875
1 Ayam Geprek tanpa nasi Cabe 0 
6 Nasi Ayam Crisbar 
2 Nasi Goreng Kulit 
7 Nasi Ayam Teriyaki 
1 Nasi Goreng Telor 
2 Nasi Ayam Richeese 
1 Nasi Goreng Spesial 
1 Indomie Ayam Geprek Cabe 1 
1 Indomie Telur Geprek Cabe 0 
1 Kulit Ayam Crispy tanpa bumbu 
1 Kulit Ayam Crispy 
1 Tahu Crispy 
2 Es Teh Jumbo 
2 Es Teh 
2 Sambal Bawang 


In [383]:
# payload = {'message' : '\n'.join(combined_prints)}
# r = requests.post('https://notify-api.line.me/api/notify'
#                 , headers={'Authorization' : 'Bearer {}'.format(token)}
#                 , params = payload)

In [384]:
# duar
# payload = {'message' : 'Duar'}
# files = {'imageFile': open('duar.jpg', 'rb')}
# r = requests.post('https://notify-api.line.me/api/notify'
#                 , headers={'Authorization' : 'Bearer {}'.format(token)}
#                 , params = payload, files=files)

In [385]:
# ambatukam
# payload = {'message' : 'Ambatukam @Jason'}
# files = {'imageFile': open('ambatukam.jpg', 'rb')}

# r = requests.post('https://notify-api.line.me/api/notify'
#                 , headers={'Authorization' : 'Bearer {}'.format(token)}
#                 , params = payload, files=files)